In [ ]:
#| default_exp markdown.obsidian.personal.notation.in_standard_information_note

# markdown.obsidian.personal.notation.in_standard_information_note
> For functions concerning notation notes as they are related to
> their "main" notes, which are stadnard information notes.

In [ ]:
#| export

import warnings
import os
from os import PathLike
from typing import Optional, Union
from pathlib import Path

from trouver.helper.definition_and_notation import notation_asterisk_indices
from trouver.helper.html import remove_html_tags_in_text 
from trouver.helper.latex import latex_indices 
from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum
from trouver.markdown.obsidian.links import ObsidianLink
from trouver.markdown.obsidian.personal.information_notes import bulleted_links_of_type_in_section
from trouver.markdown.obsidian.personal.note_type import note_is_of_type, PersonalNoteTypeEnum
from trouver.markdown.obsidian.personal.notation.parse import main_of_notation
from trouver.markdown.obsidian.vault import VaultNote, NoteDoesNotExistError

In [ ]:
from unittest import mock
import tempfile
import shutil
from fastcore.test import *
from trouver.helper.tests import _test_directory


# Notations in a standard information note

## Find notations introduced in note

#### By double asterisks `**`

**WARNING**: The use of double asterisks `**` to signify definitions and notations is deprecated.

In the legacy method, string surrounded by double asterisks `**` were recognized as marking notations

In [ ]:
#| export
def notat_str_from_doub_asts_in_std_info_note(
        info_note: VaultNote
        ) -> list[str]: # Each str is a LaTeX str, beginning and trailing dollar signs `$` (single or double) included.
    """
    Return the LaTeX str's with notations in a standard information note.

    For this function, A LaTeX str is deemed to be a notation if it is surrounded by double
    asterisks `**`
    """
    mf = MarkdownFile.from_vault_note(info_note)
    notations = []
    for part in mf.parts:
        indices = notation_asterisk_indices(part['line'])
        notations.extend([
            part['line'][start+2:end-2] for start, end in indices])
    return notations

We can obtain the notation str in an information note:

In [ ]:
vault = _test_directory() / 'test_vault_7'
# A note with just one notation:
vn = VaultNote(vault, name='galois_group')
sample_output_1 = notat_str_from_doub_asts_in_std_info_note(vn)
assert len(sample_output_1) == 1
assert sample_output_1[0].startswith('$')
assert sample_output_1[0].endswith('$')
assert not sample_output_1[0].startswith('$$')
assert not sample_output_1[0].endswith('$$')
print(sample_output_1)

# A note with a notation with double asterisks:
vn = VaultNote(vault, name='spectrum_of_a_ring')
sample_output_2 = notat_str_from_doub_asts_in_std_info_note(vn)
assert len(sample_output_2) == 2
assert sample_output_2[1].startswith('$$')
assert sample_output_2[1].endswith('$$')
print(sample_output_2)

# A note with no notations:
assert not notat_str_from_doub_asts_in_std_info_note(VaultNote(vault, name='no_notations_note_about_integral_domains'))

['$\\operatorname{Gal}(L/K)$']
['$\\operatorname{Spec} A$', '$$D(f) = \\{\\mathfrak{p} \\in \\operatorname{Spec} A: f \\not\\in \\mathfrak{p} \\}.$$']


In [ ]:
# hide
# Test notation str with asterisks.
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='direct_and_inverse_images_of_sheaves')
sample_output = notat_str_from_doub_asts_in_std_info_note(vn)
# sample_output
assert len(sample_output) == 2

#### By HTML tags

Now, HTML tags are used to mark notations

In [ ]:
#| export
def notat_str_from_html_tags(
        info_note: VaultNote
        ) -> list[tuple[str, str]]: # Each str is a LaTeX str, the first of which is the text of and surrounded by the HTML tag and the second of which is a string (without surrounding dollar signs) specifying the actual notation introduced in the first text.
    r"""
    Return the LaTeX str's with notations in a standard information note.

    For this function, HTML tags with the `notation` attr are deemed to
    contain (newly introduced) notations. The `notation` attribute can
    have multiple strings separated by double semicolons `;;` and
    not surrounded by dollar signs `$`. 

    The following are some examples of HTML tags with the format described
    above:

    1. <span notation="">$H^i$</span>
    2. <span notation="h^i">$h^i := \dim_k H^i$</span>
    3. <span notation="IJ;;I+J">$$IJ = \langle ab: a \in I, b \in J \rangle, \quad I+J = \{a+b: a \in I, b \in J \}$</span>

    **Raises**
    - UserWarning
        - If an HTML tag with a `notation` attr does not surround a pure
          math mode string.
    """
    text = info_note.text()
    _, tag_data = remove_html_tags_in_text(text)
    pairs = []
    for tag, _, _ in tag_data:
        if not 'notation' in tag.attrs:
            continue
        notat_strs = tag.attrs['notation'].split(';;')
        pairs.extend([(tag.text, notat_str) for notat_str in notat_strs])
        
        if latex_indices(tag.text) != [(0, len(tag.text))]:
            warnings.warn(
                rf"""In the note {info_note.name} at {info_note.path()},
                there is a notation {tag.text}, but it is not
                recognized as a pure LaTeX math mode string""")
    return pairs

In [ ]:
text = 'Let $K$ be a field. An <b definition="Abelian variety over a field">Abelian variety over $K$</b> is a variety that'
with mock.patch('__main__.VaultNote.text') as mock_vaultnote_text:
    mock_vaultnote_text.return_value = text
    listy = notat_str_from_html_tags(
        VaultNote('', ''))
    assert not listy

In [ ]:
text = 'Given an ordering, we write <span notation="">$<$</span> for the less-than symbol'
with mock.patch('__main__.VaultNote.text') as mock_vaultnote_text:
    mock_vaultnote_text.return_value = text
    listy = notat_str_from_html_tags(
        VaultNote('', ''))
    test_eq(listy, [('$<$', '')])

## Notation notes listed in see also section

In [ ]:
#| export
def notation_notes_linked_in_see_also_section(
        info_note: VaultNote,
        vault: PathLike, # Path to the vault directory.
        as_vault_notes: bool = True # If `True`, returns each notation note as a `VaultNote` object.  Otherwise, returns the name of each notation note. Defaults to `True`.
        ) -> Union[list[VaultNote], list[str]]: # Each entry corresponds to a notation note in the vault.
    """Return a list of notation notes listed in the
    `See Also` section of the standard information note.

    In the current implementation of this function, only 
    "notation notes" that actually exist are included in
    the returned list.
    """
    links = bulleted_links_of_type_in_section(
        info_note, vault, section="See Also",
        note_type=PersonalNoteTypeEnum.NOTATION_NOTE)
    note_names = [link.file_name for link in links]
    if as_vault_notes:
        return [VaultNote(vault, name=note_name, update_cache=False) for note_name in note_names]
    else:
        return note_names


The `notation_notes_linked_in_see_also_section` method detects the Notation notes listed in bulleted links in the See Also section of a standard information note.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='twist_of_a_graded_module')
sample_output = notation_notes_linked_in_see_also_section(vn, vault)
assert len(sample_output) == 1
assert isinstance(sample_output[0], VaultNote)
print(sample_output[0].name)
assert sample_output[0].exists()

foag_notation_M_n_bullet


Setting `as_vault_notes=False` returns the names of the notation notes.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='twist_of_a_graded_module')
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=False)
assert len(sample_output) == 1
assert isinstance(sample_output[0], str)
print(sample_output[0])

foag_notation_M_n_bullet


If a "notation note" does not exist, then it is not included in the returned list.

Note that `notation_notes_linked_in_see_also_section` uses `bulleted_links_of_type_in_section`, which is turn uses `note_is_of_type` to get a list of linked notes of the type `NOTATION_NOTE` of the `PersonalNoteTypeEnum` class. In turn, a note file must exist for the note to be considered of any particular type under the current implementation of `note_is_of_type`.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='note_with_links_to_non_existent_notation_notes')
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=True)
test_eq(sample_output, [])
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=False)
test_eq(sample_output, [])

## Find all notation notes in a vault subdirectory

In [ ]:
#| export 
def notations_and_main_notes(
        vault: PathLike, # Path to the vault directory.
        subdirectory: Optional[PathLike] = None, # Path to the subdirectory, relative to `vault`, to find the notation notes. Searches for all notation notes here and in subdirectories of this subdirectory. If `None`, then the `note parameter is used to determined the subdirectory. If `subdirectory` is the empty str, then all notation notes in the vault are searched. Defaults to `None`. 
        note: Optional[VaultNote] = None # A note in the vault. The directory that this note is in determines the `subdirectory` parameter if the argument passed to `subdirectory` is the blank str. This note can usually be an index note, e.g. `'_index_silverman'`. Defaults to `None`, in which case `subdirectory` must be specified.
        ) -> dict[str, Union[str, None]]: # A key is the unique name of a notation note in the vault and its corresponding value is the name of the main note of the notation note. Each main note may not actually exist, but each notation note definitely exists. If the notation note has no main note (i.e. has no links to other notes), then the value is `None`.
    """Return a `dict` with all of notation notes in a specified
    subdirectory of a vault and the names of the main notes of these
    notation notes.
    
    **Returns**
    - `dict`

    **Raises**

    - ValueError
        - If `subdirectory` and `note` are both `None`.
    """
    if note is None and subdirectory is None:
        raise ValueError(
            'Both the `subdirectory` and `note` parameters are None.')
    # vault = vault if vault is not None else ''
    if subdirectory is None:
        subdirectory = Path(note.rel_path).parent
    subdirectory_path = Path(vault) / subdirectory
    notes_in_subdirectory = subdirectory_path.glob(f'**/*.md') 
    relative_paths = [Path(note_path).relative_to(subdirectory_path)
                      for note_path in notes_in_subdirectory]
    vn_objects = [VaultNote(vault, rel_path=Path(subdirectory) / rel_path)
                  for rel_path in relative_paths]
    return {vn.name: main_of_notation(vn) for vn in vn_objects
            if note_is_of_type(vn, PersonalNoteTypeEnum.NOTATION_NOTE)}

The `notations_and_main_notes` function returns all the notation notes in a subdirectory of a vault

In [ ]:
vault = _test_directory() / 'test_vault_7'
sample_output = notations_and_main_notes(vault, subdirectory='')
print(sample_output)
assert 'foag_notation_M_n_bullet' in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' in sample_output
assert sample_output['some_reference_name_notation_O_X_this_file_has_no_links'] is None
assert 'poonen_curves_notation_Z_X_T' in sample_output

{'foag_notation_M_n_bullet': 'twist_of_a_graded_module', 'notation_note_with_main_note_link_but_main_note_does_not_exist': 'nonexistent_note', 'some_reference_name_notation_k_t_formal_power_series_ring': 'some_note', 'some_reference_name_notation_O_X_this_file_has_no_links': None, 'some_reference_name_notation_Pic_C': 'divisor_class_group_of_a_curve', 'some_reference_name_notation_Spec_A': 'spectrum_of_a_ring', 'foag_notation_O_n': 'foag_15.2.1', 'foag_notation_O_text_Proj__S__n': 'foag_15.2.1', 'poonen_curves_notation_zeta_X_s_zeta_function_of_variety': 'poonen_curves_3.4.1 DEFINITION', 'poonen_curves_notation_Z_X_T': 'poonen_curves_3.4.1 DEFINITION', 'some_reference_name_notation_B_R': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_B_R_1': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_Jac_C': 'note_with_some_excessive_notation_notes', 'test_notation_note_for_latex_in_original_metadata_1': 'test_note_for_latex_in_original_metadata',

Here is an example with a subdirectory specified:

In [ ]:
vault = _test_directory() / 'test_vault_7'
sample_output = notations_and_main_notes(vault, subdirectory='some_other_folder')
print(sample_output)
assert 'foag_notation_M_n_bullet' not in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' not in sample_output
assert 'poonen_curves_notation_Z_X_T' in sample_output

{'foag_notation_O_n': 'foag_15.2.1', 'foag_notation_O_text_Proj__S__n': 'foag_15.2.1', 'poonen_curves_notation_zeta_X_s_zeta_function_of_variety': 'poonen_curves_3.4.1 DEFINITION', 'poonen_curves_notation_Z_X_T': 'poonen_curves_3.4.1 DEFINITION', 'some_reference_name_notation_B_R': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_B_R_1': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_Jac_C': 'note_with_some_excessive_notation_notes'}


Alternatively, we can specify a subdirectory by a `VaultNote` object; the directory that the `VaultNote` object is the subdirectory:

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='galois_group')
sample_output = notations_and_main_notes(vault, subdirectory=None, note=vn)
print(sample_output)
assert 'foag_notation_M_n_bullet' in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' in sample_output
assert sample_output['some_reference_name_notation_O_X_this_file_has_no_links'] is None
assert 'poonen_curves_notation_Z_X_T' not in sample_output

{'foag_notation_M_n_bullet': 'twist_of_a_graded_module', 'notation_note_with_main_note_link_but_main_note_does_not_exist': 'nonexistent_note', 'some_reference_name_notation_k_t_formal_power_series_ring': 'some_note', 'some_reference_name_notation_O_X_this_file_has_no_links': None, 'some_reference_name_notation_Pic_C': 'divisor_class_group_of_a_curve', 'some_reference_name_notation_Spec_A': 'spectrum_of_a_ring'}


## Check if there is a notation link in the `See Also` section of a standard information note.

In [ ]:
#| export
def notation_note_is_linked_in_see_also_section(
        notation_note: VaultNote,
        info_note: Optional[VaultNote] = None # The note in which to find the link to `notation_note`. Defaults to `None`, in which case the main note is determined to be the first linked note of `notation_note`.
        ) -> bool:
    """Return `True` if a notation note is linked in the `See Also`
    section of a standard information note. 
    """
    if not info_note:
        info_note = main_of_notation(notation_note, as_note=True)
    notes = notation_notes_linked_in_see_also_section(
        info_note, vault=notation_note.vault, as_vault_notes=False)
    return notation_note.name in notes


`notation_note_is_linked_in_see_also_section` Returns `True` if a notation note is linked in the `See Also` section of an information note:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Pic_C')
info_note = VaultNote(vault, name='divisor_class_group_of_a_curve')
assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)

notation_note = VaultNote(vault, name='foag_notation_M_n_bullet')
info_note = VaultNote(vault, name='twist_of_a_graded_module')
assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

If `info_note` is not specified or if `info_note=None`, then the info note in question is the first note which the notation note links to, see the `main_of_notation` function:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Pic_C')
assert not notation_note_is_linked_in_see_also_section(notation_note)

notation_note = VaultNote(vault, name='foag_notation_M_n_bullet')
assert notation_note_is_linked_in_see_also_section(notation_note)

## Add notation links to the `See Also` section

#### Add links to individual notation notes

In [ ]:
#| export
def add_notation_note_to_see_also(
        notation_note: VaultNote,
        info_note: Optional[VaultNote] = None, # The note in which to link `notation_note`. Defaults to `None`, in which case the main note is determined to be the first linked note of `notation_note`.
        do_not_repeat: bool = True # If `True`, do not add a link to `notation_note` in if there is already a such a link.
        ) -> None:
    """Add a link to a notation note in the `See Also` section of
    a standard information note.

    **Raises**

    - NoteDoesNotExistError
        - If the information note to link to does not exist.
    
    """
    if not info_note:
        info_note = main_of_notation(notation_note, as_note=True)
    if not info_note.exists():
        raise NoteDoesNotExistError
    if do_not_repeat and notation_note_is_linked_in_see_also_section(
            notation_note, info_note):
        return
    mf = MarkdownFile.from_vault_note(info_note)
    link = ObsidianLink(False, notation_note.name, 0, 0)
    mf.add_line_in_section(
        'See Also', {'type': MarkdownLineEnum.UNORDERED_LIST,
                     'line': f'- {str(link)}'})
    mf.write(info_note)



The `add_notation_note_to_see_also` method adds a link to the specified notation note to the specified standard information note.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    info_note = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)    
    print("The note's text before adding the link:")
    print(info_note.text(), '\n\n')

    add_notation_note_to_see_also(notation_note, info_note)
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    print("The note's text after adding the link:")
    print(info_note.text())

The note's text before adding the link:
---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Divisor class group of a curve[^1]
Let $C/k$ be a curve.

The **divisor class group** of $C$, denoted **$\operatorname{Pic} C$**, is defined as $\operatorname{Div} C / \operatorname{Princ} C$.  

# See Also

# Meta
## References

## Citations and Footnotes
[^1]: Citation
 


The note's text after adding the link:
---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Divisor class group of a curve[^1]
Let $C/k$ be a curve.

The **divisor class group** of $C$, denoted **$\operatorname{Pic} C$**, is defined as $\operatorname{Div} C / \operatorname{Princ} C$.  

# See Also
- [[some_reference_name_notation_Pic_C]]

# Meta
## References

## Citations and Footnotes
[^1]: Citation



If `info_note` is not specified or is `None`, then the information note to add the link is determined to be the first note that `notation_note` links to:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    assert not notation_note_is_linked_in_see_also_section(notation_note)
    add_notation_note_to_see_also(notation_note)
    assert notation_note_is_linked_in_see_also_section(notation_note)

If the information note to add the link does not exist, then a `NoteDoesNotExistError` is raised:

If `do_not_repeat` is not specified or is `True`, then the link to the notation note is added only if such a link is not already present: 

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='foag_notation_M_n_bullet')
    info_note = VaultNote(temp_vault, name='twist_of_a_graded_module')
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    original_text = info_note.text()
    add_notation_note_to_see_also(notation_note, do_not_repeat=True)
    new_text = info_note.text()
    assert original_text == new_text
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

    # notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    # add_notation_note_to_see_also(notation_note)
    # assert notation_note_is_linked_in_see_also_section(notation_note)

Otherwise, the link to the notation note is added even if such a link is already present:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='foag_notation_M_n_bullet')
    info_note = VaultNote(temp_vault, name='twist_of_a_graded_module')
    original_text = info_note.text()
    add_notation_note_to_see_also(notation_note, do_not_repeat=False)
    new_text = info_note.text()
    assert original_text != new_text
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    print(new_text)


---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note]
---
# Twist of graded module[^1]

Suppose $M_\bullet$ is a graded $S_\bullet$-module. Define the graded module **$M(n)_\bullet$** by $M(n)_{m}:=M_{n+m}$. Thus the quasicoherent sheaf $M(n)_\bullet$ satisfies

$$ \Gamma\left(D(f), \widetilde{M(n)}_{\bullet}\right)=\left(\left(M_{\bullet}\right)_{f}\right)_{n} $$

where here the subscript means we take the nth graded piece.

# See Also
- [[foag_notation_M_n_bullet]]
- [[foag_notation_M_n_bullet]]
# Meta
## References
![[_reference_foag]]

## Citations and Footnotes
[^1]: Vakil, Invertible 15.2, Page 412



If a link to the notation note is not already present, then `add_notation_note_to_see_also` adds such a link to the info note whether or not `do_not_repeat` is `True`:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    info_note = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)    

    add_notation_note_to_see_also(notation_note, info_note, do_not_repeat=False)
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

#### Add links to notation notes for all notation notes in a specified subdirectory.

In [ ]:
#| export
def add_missing_notation_links_to_information_notes(
        vault: PathLike, # Path to the vault directory.
        subdirectory: Optional[PathLike] = None, # Path to the subdirectory, relative to `vault`, to find the notation notes and their main notes. Searches for all notation notes here and in subdirectories of this subdirectory. If `None`, then the `note` parameter is used to determine `subdirectory`. Defaults to `None`. 
        note: Optional[VaultNote] = None # A note in the vault. The directory that this note is in determines the `subdirectory` parameter if it is `None`.  Defaults to `None`, in which case `subdirectory` must be specified.
        ) -> None:
    # TODO: deal with possibility that note does not exist.
    """For each notation note in a specified subdirectory, Add links
    to notation notes in their main information notes if the notation
    links are not already present.
    
    **Raises**

    - ValueError
        - If `subdirectory` and `note` are both `None`.
    """
    mains_dict = notations_and_main_notes(vault, subdirectory, note)
    to_check = {key: value for key, value in mains_dict.items()
                if value is not None}
    for notation, main in to_check.items():
        if not main:
            continue
        notation_note = VaultNote(vault, name=notation)
        try:
            main_note = VaultNote(vault, name=main)  # TODO add the subdirectory parameter here appropriately. See Also `notations_and_main_note` 
            add_notation_note_to_see_also(notation_note, main_note)
        except NoteDoesNotExistError:
            continue

The `add_missing_notation_links_to_information_notes` method adds links to notation notes of a specified directory to their main notes if these links are not already present. Note that nonexisting information notes which are the "main" notes of notation notes are ignored.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note_1 = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    main_note_1 = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    notation_note_2 = VaultNote(temp_vault, name='some_reference_name_notation_Spec_A')
    main_note_2 = VaultNote(temp_vault, name='spectrum_of_a_ring')
    notation_note_3 = VaultNote(temp_vault, name='foag_notation_O_n')
    main_note_3 = VaultNote(temp_vault, name='foag_15.2.1')

    assert not notation_note_is_linked_in_see_also_section(notation_note_1, main_note_1)
    assert not notation_note_is_linked_in_see_also_section(notation_note_2, main_note_2)
    assert not notation_note_is_linked_in_see_also_section(notation_note_3, main_note_3)
    add_missing_notation_links_to_information_notes(temp_vault, '')
    assert notation_note_is_linked_in_see_also_section(notation_note_1, main_note_1)
    assert notation_note_is_linked_in_see_also_section(notation_note_2, main_note_2)
    assert notation_note_is_linked_in_see_also_section(notation_note_3, main_note_3)